In [1]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, GRU, Activation
from keras.callbacks import TensorBoard
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.preprocessing import StandardScaler

Using TensorFlow backend.


In [2]:
df = pd.read_csv('../train.csv')
print(df.shape)
df.head()

(10886, 12)


,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,13
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1


In [3]:
df_train = df.copy()
# df_train['month'] = pd.DatetimeIndex(df_train['datetime']).month
df_train['day'] = pd.DatetimeIndex(df_train['datetime']).day
df_train['dayofweek'] = pd.DatetimeIndex(df_train['datetime']).dayofweek
df_train['hour'] = pd.DatetimeIndex(df_train['datetime']).hour
df_train = df_train.drop('datetime', axis=1)
y_casual = df_train['casual'].copy()
y_registered = df_train['registered'].copy()
df_train = df_train.drop(['casual','registered','count'], axis=1)
x_casual = df_train.copy()
x_registered = df_train.copy()
x_casual['casual_rush'] = x_casual[['hour', 'workingday']].apply(
    lambda x: (0, 1)[
        (x['workingday'] == 1 and  11 <= x['hour'] <= 20)
        or (x['workingday'] == 0 and  10 <= x['hour'] <= 19)], axis = 1)
x_registered['registered_rush'] = x_registered[['hour', 'workingday']].apply(
    lambda x: (0, 1)[
        (x['workingday'] == 1 and  (6 <= x['hour'] <= 9 or 16 <= x['hour'] <= 21))
        or (x['workingday'] == 0 and  9 <= x['hour'] <= 22)], axis = 1)
print(x_casual.shape, x_registered.shape)
x_casual.head()

(10886, 12) (10886, 12)


,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,day,dayofweek,hour,casual_rush
0,1,0,0,1,9.84,14.395,81,0.0,1,5,0,0
1,1,0,0,1,9.02,13.635,80,0.0,1,5,1,0
2,1,0,0,1,9.02,13.635,80,0.0,1,5,2,0
3,1,0,0,1,9.84,14.395,75,0.0,1,5,3,0
4,1,0,0,1,9.84,14.395,75,0.0,1,5,4,0


In [29]:
X = x_registered.values
Y = y_registered.values
n_features = X.shape[1]
scaler = StandardScaler()
X = scaler.fit_transform(X)
X = X.reshape(X.shape[0], 1, n_features)
def new_model():
    # create model
    model = Sequential()
    model.add(Dense(100, activation='relu'))
    model.add(GRU(50, input_shape=(1, n_features), return_sequences=True, activation='relu'))
    model.add(GRU(50, input_shape=(1, n_features), activation='relu'))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(1))
    model.add(Activation('linear'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='RMSprop')
    return model
kr_registered = KerasRegressor(build_fn=new_model, nb_epoch=100, batch_size=5, verbose=2)
# %time kr_casual.fit(X, Y, callbacks=[TensorBoard(log_dir='logs')])
%time kr_registered.fit(X, Y)
X = x_casual.values
Y = y_casual.values
n_features = X.shape[1]
X = scaler.fit_transform(X)
X = X.reshape(X.shape[0], 1, n_features)
kr_casual = KerasRegressor(build_fn=new_model, nb_epoch=100, batch_size=5, verbose=2)
%time kr_casual.fit(X, Y)

Exception: The first layer in a Sequential model must get an `input_shape` or `batch_input_shape` argument.

## Prediction

In [5]:
df_test = pd.read_csv('../test.csv')
df_test.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed
0,2011-01-20 00:00:00,1,0,1,1,10.66,11.365,56,26.0027
1,2011-01-20 01:00:00,1,0,1,1,10.66,13.635,56,0.0000
2,2011-01-20 02:00:00,1,0,1,1,10.66,13.635,56,0.0000
3,2011-01-20 03:00:00,1,0,1,1,10.66,12.880,56,11.0014
4,2011-01-20 04:00:00,1,0,1,1,10.66,12.880,56,11.0014


In [27]:
dft = df_test.copy()
# dft['month'] = pd.DatetimeIndex(dft['datetime']).month
dft['day'] = pd.DatetimeIndex(dft['datetime']).day
dft['dayofweek'] = pd.DatetimeIndex(dft['datetime']).dayofweek
dft['hour'] = pd.DatetimeIndex(dft['datetime']).hour
dft = dft.drop('datetime', axis=1)
t_casual = dft.copy()
t_registered = dft.copy()
t_casual['casual_rush'] = t_casual[['hour', 'workingday']].apply(
    lambda x: (0, 1)[
        (x['workingday'] == 1 and  11 <= x['hour'] <= 20)
        or (x['workingday'] == 0 and  10 <= x['hour'] <= 19)], axis = 1)
t_registered['registered_rush'] = t_registered[['hour', 'workingday']].apply(
    lambda x: (0, 1)[
        (x['workingday'] == 1 and  (6 <= x['hour'] <= 9 or 16 <= x['hour'] <= 21))
        or (x['workingday'] == 0 and  9 <= x['hour'] <= 22)], axis = 1)
print(t_casual.shape, t_registered.shape)
t_casual.head()

(6493, 12) (6493, 12)


,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,day,dayofweek,hour,casual_rush
0,1,0,1,1,10.66,11.365,56,26.0027,20,3,0,0
1,1,0,1,1,10.66,13.635,56,0.0000,20,3,1,0
2,1,0,1,1,10.66,13.635,56,0.0000,20,3,2,0
3,1,0,1,1,10.66,12.880,56,11.0014,20,3,3,0
4,1,0,1,1,10.66,12.880,56,11.0014,20,3,4,0


In [28]:
t_casual = scaler.fit_transform(t_casual)
t_registered = scaler.fit_transform(t_registered)
t_casual = t_casual.reshape(t_casual.shape[0], 1, n_features)
t_registered = t_registered.reshape(t_registered.shape[0], 1, n_features)
test_pred_casual = kr_casual.predict(t_casual)
test_pred_casual[test_pred_casual < 0] = 0
test_pred_registered = kr_registered.predict(t_registered)
test_pred_registered[test_pred_registered < 0] = 0
test_pred = test_pred_casual + test_pred_registered
str_prediction = "datetime,count\n"
for i in range(0, len(test_pred)):
    datetime = df_test['datetime'][i]
    pred = int(round(test_pred[i]))
    str_prediction += "{},{}\n".format(datetime, pred)

# print(str_prediction)
f = open('pred_keras_gru.csv','w')
f.write(str_prediction)
f.close()